# Install important libraries

In [ ]:
!pip install ultralytics
!pip install pillow
!pip install transformers verovio tiktoken

In [ ]:
from sqlalchemy import create_engine, Column, String, Integer, Boolean, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from transformers import AutoModel, AutoProcessor
from PIL import Image, ImageDraw, ImageFont
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
import matplotlib.pyplot as plt
from datetime import datetime
from ultralytics import YOLO
from email import encoders
import smtplib
import sqlite3
import torch
import os
import re







# Setup Database 

## Motorbike Detections Database

In [ ]:
# Function to print all the records from 'MotorbikeDetections'
def print_motorbike_detections():
    # Connect to the motorbike detections database
    conn = sqlite3.connect('motorbike_detections.db')
    cursor = conn.cursor()

    # Query all records from the MotorbikeDetections table
    cursor.execute('SELECT * FROM MotorbikeDetections')
    rows = cursor.fetchall()

    # Print headers
    print(f"{'ID':<3} {'Date':<12} {'Time':<8} {'City':<10} {'License Plate':<15} {'Lane Viol.':<11} {'No Helmet':<10} {'Image Link'}")

    # Print each row
    for row in rows:
        id, date, time, city, license_plate, lane_vio, no_helmet, img_link = row
        lane_vio = 'True' if lane_vio else 'False'
        no_helmet = 'True' if no_helmet else 'False'
        print(f"{id:<3} {date:<12} {time:<8} {city:<10} {license_plate:<15} {lane_vio:<11} {no_helmet:<10} {img_link}")

    # Close the connection
    conn.close()

# Function to print all the records from 'VehicleInformation'
def print_vehicle_information():
    # Connect to the vehicle information database using SQLAlchemy
    engine = create_engine('sqlite:///vehicle_information.db')
    Session = sessionmaker(bind=engine)
    session = Session()

    # Query all records from the VehicleInformation table
    vehicle_info_list = session.query(VehicleInformation).all()

    # Print headers
    print(f"{'License Plate':<15} {'Email':<25} {'Phone Number':<15} {'Driver ID'}")

    # Print each vehicle record
    for vehicle in vehicle_info_list:
        print(f"{vehicle.license_plate:<15} {vehicle.email:<25} {vehicle.phone_number:<15} {vehicle.driver_id}")

    # Close the session
    session.close()


## Vehicle Information Database

In [ ]:


def get_vehicle_information(detected_license_plate, lane_violation, no_helmet, image_link, city):
    # Get current date and time
    current_datetime = datetime.now()
    current_date = current_datetime.strftime('%Y-%m-%d')
    current_time = current_datetime.strftime('%H:%M:%S')

    # Connect to SQLite database (motorbike_detections.db)
    try:
        motorbike_conn = sqlite3.connect('motorbike_detections.db')
        motorbike_cursor = motorbike_conn.cursor()

        # Check if the detection already exists
        check_query = '''
            SELECT DetectionID FROM MotorbikeDetections
            WHERE LicensePlate = ? AND Date = ? AND Time = ? AND ImageLink = ?
        '''
        motorbike_cursor.execute(check_query, (detected_license_plate, current_date, current_time, image_link))
        existing_detection = motorbike_cursor.fetchone()

        if existing_detection:
            print(f"Detection for license plate {detected_license_plate} at {current_date} {current_time} already exists.")
        else:
            # Insert the new detection record
            insert_query = '''
                INSERT INTO MotorbikeDetections (Date, Time, City, LicensePlate, LaneViolation, NoHelmet, ImageLink)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            '''
            motorbike_cursor.execute(insert_query, (
                current_date,
                current_time,
                city,
                detected_license_plate,
                int(lane_violation),  # Convert boolean to integer (1 or 0)
                int(no_helmet),        # Convert boolean to integer (1 or 0)
                image_link
            ))

            # Commit the transaction
            motorbike_conn.commit()
            print(f"Detection data for license plate {detected_license_plate} inserted successfully.")

    except sqlite3.IntegrityError as e:
        print(f"Integrity Error: {e}. This detection may already exist.")
    except sqlite3.Error as e:
        print(f"An error occurred while inserting detection data: {e}")
        motorbike_conn.rollback()
    finally:
        # Close the motorbike detections database connection
        motorbike_conn.close()

    # Retrieve email from 'vehicle_information.db'
    try:
        # Create an engine and session for SQLAlchemy
        engine = create_engine('sqlite:///vehicle_information.db')
        Session = sessionmaker(bind=engine)
        session = Session()

        # Query the VehicleInformation table for the detected license plate
        vehicle_info = session.query(VehicleInformation).filter_by(license_plate=detected_license_plate).first()

        if vehicle_info:
            print(f"Email found for license plate {detected_license_plate}: {vehicle_info.email}")
            return vehicle_info.email
        else:
            print(f"No vehicle information found for license plate {detected_license_plate}.")
            return None

    except Exception as e:
        print(f"An error occurred while retrieving vehicle information: {e}")
        return None

    finally:
        # Close the SQLAlchemy session
        session.close()


## Toy Example for sending email

In [9]:
# Example Usage
if __name__ == "__main__":
    # Sample data
    detected_license_plate = '1234 AB'
    lane_violation = False
    no_helmet = True
    image_link = 'path_to_image/image1.png'
    city = 'Riyadh'

# Call the function
email = get_vehicle_information(detected_license_plate, lane_violation, no_helmet, image_link, "Riyadh")
if email:
    print(f"Send notification to: {email}")
else:
    print("No email found to send notification.")

Detection data for license plate 1234AB inserted successfully.
Email found for license plate 1234AB: nalsaqer67@gmail.com
Send notification to: nalsaqer67@gmail.com


## Print Database

In [27]:
# Example usage:
# Print all records in the MotorbikeDetections database
print("\nMotorbike Detections Database:")
print_motorbike_detections()

# Print all records in the VehicleInformation database
print("\nVehicle Information Database:")
print_vehicle_information()


Motorbike Detections Database:
ID  Date         Time     City       License Plate   Lane Viol.  No Helmet  Image Link
1   2024-09-29   15:22:00 Riyadh     1234 AB         False       True       path_to_image/image1.png
2   2024-09-30   10:45:00 Jeddah     5678 CD         True        False      path_to_image/image2.png
3   2024-09-30   14:10:00 Dammam     9101 EF         False       False      path_to_image/image3.png

Vehicle Information Database:
License Plate   Email                     Phone Number    Driver ID
1234AB          nalsaqer67@gmail.com      0559947203      ID1110000000
3321AR99        fares5675@gmail.com       0539003545      ID2220000000
3321AR          fares5675@gmail.com       0539003545      ID2220000000
1234 AB         nalsaqer67@gmail.com      0559947203      ID1110000000
3321 AR         fares5675@gmail.com       0539003545      ID2220000000


# Setup Functions & Variables

In [ ]:


# Email credentials for sending emails (replace with your App-Specific Password)
FROM_EMAIL = "Fares5675@gmail.com"
EMAIL_PASSWORD = "cawxqifzqiwjufde"  # Use App-Specific Password here
TO_EMAIL = "kduwaysan@gmail.com"
SMTP_SERVER = 'smtp.gmail.com'
SMTP_PORT = 465

# Arabic dictionary for converting license plate text
arabic_dict = {
    "0": "٠", "1": "١", "2": "٢", "3": "٣", "4": "٤", "5": "٥",
    "6": "٦", "7": "٧", "8": "٨", "9": "٩", "A": "ا", "B": "ب",
    "J": "ح", "D": "د", "R": "ر", "S": "س", "X": "ص", "T": "ط",
    "E": "ع", "G": "ق", "K": "ك", "L": "ل", "Z": "م", "N": "ن",
    "H": "ه", "U": "و", "V": "ي", " ": " "
}
class_colors = {
    0: (0, 255, 0),    # Green (Helmet)
    1: (255, 0, 0),    # Blue (License Plate)
    2: (0, 0, 255),    # Red (MotorbikeDelivery)
    3: (255, 255, 0),  # Cyan (MotorbikeSport)
    4: (255, 0, 255),  # Magenta (No Helmet)
    5: (0, 255, 255),  # Yellow (Person)
}

# Load the OCR model
processor = AutoProcessor.from_pretrained("stepfun-ai/GOT-OCR2_0", trust_remote_code=True)
model_ocr = AutoModel.from_pretrained("stepfun-ai/GOT-OCR2_0", trust_remote_code=True).to('cuda')

# Load YOLO model
model = YOLO('/kaggle/input/private-dataset/yolov8_Medium.pt')

# Define lane area coordinates (example coordinates)
red_lane = np.array([[2,1583],[1,1131],[1828,1141],[1912,1580]], np.int32)

# Path for Arabic font
font_path = "/kaggle/input/fontss/alfont_com_arial-1.ttf"

# Paths for saving violation images
violation_image_path = '/kaggle/working/violation.jpg'

# Track emails already sent to avoid duplicate emails
sent_emails = {}

# Dictionary to track violations per license plate
violations_dict = {}

# Video path (input)
video_path = "/kaggle/input/video-tests/IMG_2095_2.mp4"

In [ ]:
# Function to filter license plate text
def filter_license_plate_text(license_plate_text):
    '''
    objective:
    This function filters and extracts a license plate string by removing non-alphanumeric characters
    and then extracting a pattern that consists of four digits followed by two uppercase letters.

    Args:
    license_plate_text (str): A string representing the raw license plate text which may contain
    non-alphanumeric characters.

    Returns:
    str: A formatted string containing four digits followed by two uppercase letters, separated by
    a space (e.g., '1234 AB').
    '''
    license_plate_text=re.sub(r'[^A-Z0-9]+',"",license_plate_text)
    match = re.search(r'(\d{4})\s*([A-Z]{2})', license_plate_text)
    return f"{match.group(1)} {match.group(2)}" if match else None



# Function to convert text to Arabic
def convert_to_arabic(license_plate_text):
    '''
    Brief objective:
    Converts a given license plate text (in English) into Arabic characters using a predefined dictionary.

    Args:
    license_plate_text (str): A string containing the license plate text in English.

    Returns:
    str: The license plate text  converted into Arabic based on the `arabic_dict` mapping. 
    If a character is not in the dictionary, it remains unchanged.
    '''
    return "".join(arabic_dict.get(char, char) for char in license_plate_text)




# Function to send email notification with image attachment
def send_email(license_text, violation_image_path, violation_type, email):
    '''
    Brief objective:
    Sends an email notification about a traffic violation, including an image attachment of the violation.

    Args:
    license_text (str): The text of the license plate, which will be included in the email body.
    violation_image_path (str): Path to the image file showing the violation.
    violation_type (str): Type of traffic violation (e.g., 'No Helmet', 'In Red Lane', 'No Helmet, In Red Lane').
    email (str): The recipient's email address.

    Returns:
    None: This function sends the email but does not return any value. It logs success or error messages.
    '''
    # Define the subject and body based on violation type
    if violation_type == 'No Helmet, In Red Lane':
        subject = 'تنبيه مخالفة: عدم ارتداء خوذة ودخول المسار الأيسر'
        body = f"لعدم ارتداء الخوذة ولدخولها المسار الأيسر ({license_text}) تم تغريم دراجة نارية التي تحمل لوحة"
        
    elif violation_type == 'In Red Lane':
        subject = 'تنبيه مخالفة: دخول المسار الأيسر'
        body = f"لدخولها المسار الأيسر ({license_text}) تم تغريم دراجة نارية التي تحمل لوحة"
    else:
        violation_type == 'No Helmet'
        subject = 'تنبيه مخالفة: عدم ارتداء خوذة'
        body = f"لعدم ارتداء الخوذة ({license_text}) تم تغريم دراجة نارية التي تحمل لوحة"
        
    # Create the email message
    msg = MIMEMultipart()
    msg['From'] = FROM_EMAIL
    msg['To'] = email
    msg['Subject'] = subject 
    msg.attach(MIMEText(body, 'plain'))

    # Attach the violation image
    if os.path.exists(violation_image_path):
        with open(violation_image_path, 'rb') as attachment_file:
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(attachment_file.read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition', f'attachment; filename={os.path.basename(violation_image_path)}')
            msg.attach(part)

    # Send the email using SMTP
    with smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT) as server:
        server.login(FROM_EMAIL, EMAIL_PASSWORD)
        server.sendmail(FROM_EMAIL, TO_EMAIL, msg.as_string())
        print("Email with attachment sent successfully!")
        
def draw_text_pil(img, text, position, font_path, font_size, color):
    '''
    Brief objective:
    Draws text on an image at a given position using the specified font, size, and color.

    Args:
    img (numpy.ndarray): The image on which to draw the text.
    text (str): The text to draw on the image.
    position (tuple): The (x, y) coordinates where the text should be placed on the image.
    font_path (str): The file path to the font to be used for the text.
    font_size (int): The size of the font.
    color (tuple): The color of the text in (R, G, B) format.

    Returns:
    numpy.ndarray: The image with the drawn text in OpenCV (BGR) format.
    '''
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    
    draw = ImageDraw.Draw(img_pil)
    
    try:
        font = ImageFont.truetype(font_path, size=font_size)
    except IOError:
        print(f"Font file not found at {font_path}. Using default font.")
        font = ImageFont.load_default()
        
    draw.text(position, text, font=font, fill=color)
    
    img_np = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
    return img_np

## Take one frame from the Video for testing

In [ ]:

video_path = "/kaggle/input/video-tests/IMG_2095_2.mp4"
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read() 
save_path = '/kaggle/working/frame_to_save.jpg'  # Define the file path and format
cv2.imwrite(save_path, frame)

# Process Full-Video

### Video infos

In [ ]:
cap = cv2.VideoCapture(video_path)

# Check if the video file opened successfully
if not cap.isOpened():
    print("Error opening video file")
    exit()

# Define codec and output video settings
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video_path = '/kaggle/working/output_violation.mp4'
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # Frame width
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # Frame height
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

margin_y = 50                       #add 50 pixels in top of motorbike detection to guarantee helmet detection


### Apply All Together: 
1. Detect motorbike delivery
2. check lane violation 
3. crop motorbike delivery bounding box to guarantee detected object belongs to that motorbike
4. check for helmet violation
5. extract license plate if there is a violation 
6. send email about the violation

In [28]:




# Process the video frame by frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # End of video

    # Draw the red lane rectangle on each frame
    cv2.polylines(frame, [red_lane], isClosed=True, color=(0, 0, 255), thickness=3)  # Red lane

    # Perform detection using YOLO on the current frame
    results = model.track(frame)
    
    # Process each detection in the results
    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())  # Bounding box coordinates
        label = model.names[int(box.cls)]  # Class name (MotorbikeDelivery, Helmet, etc.)
        color = class_colors[int(box.cls)]
        confidence = box.conf[0].item()

        # Initialize flags and variables for the violations
        helmet_violation = False
        lane_violation = False
        violation_type = []

        # Draw bounding box around detected object
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)  # 3 is the thickness of the rectangle

        # Add label to the box (e.g., 'MotorbikeDelivery')
        cv2.putText(frame, f'{label}: {confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        # Detect MotorbikeDelivery
        if label == 'MotorbikeDelivery' and confidence >= 0.4:
            motorbike_crop = frame[max(0, y1 - margin_y):y2, x1:x2]
            delivery_center = ((x1 + x2) // 2, (y2))
            in_red_lane = cv2.pointPolygonTest(red_lane, delivery_center, False)
            if in_red_lane >= 0:
                lane_violation = True
                violation_type.append("In Red Lane")
            
            

            # Perform detection within the cropped motorbike region
            sub_results = model(motorbike_crop)

            for result in sub_results[0].boxes:
                sub_x1, sub_y1, sub_x2, sub_y2 = map(int, result.xyxy[0].cpu().numpy())  # Bounding box coordinates
                sub_label = model.names[int(result.cls)]
                sub_color = (255, 0, 0)  # Red color for the bounding box of sub-objects

                # Draw bounding box around sub-detected objects (No_Helmet, License_plate, etc.)
                cv2.rectangle(motorbike_crop, (sub_x1, sub_y1), (sub_x2, sub_y2), sub_color, 2)
                cv2.putText(motorbike_crop, sub_label, (sub_x1, sub_y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, sub_color, 2)

                if sub_label == 'No_Helmet':
                    helmet_violation = True
                    violation_type.append("No Helmet")
                    continue
                if sub_label == 'License_plate':
                    license_crop = motorbike_crop[sub_y1:sub_y2, sub_x1:sub_x2]

                    # Apply OCR if a violation is detected
                    if helmet_violation or lane_violation:
                        # Perform OCR on the license plate
                        cv2.imwrite(violation_image_path, frame)
                        license_plate_pil = Image.fromarray(cv2.cvtColor(license_crop, cv2.COLOR_BGR2RGB))
                        temp_image_path = '/kaggle/working/license_plate.png'
                        license_plate_pil.save(temp_image_path)
                        license_plate_text = model_ocr.chat(processor, temp_image_path, ocr_type='ocr')
                        filtered_text = filter_license_plate_text(license_plate_text)
                        # Check if the license plate is already detected and saved
                        if filtered_text:
                            # get the email from the database
                            
                            email = get_vehicle_information(filtered_text, lane_violation, helmet_violation, violation_image_path, "Riyadh")
                        
                            # Add the license plate and its violations to the violations dictionary
                            if filtered_text not in violations_dict:
                                violations_dict[filtered_text] = violation_type  #{"1234AB":[no_Helmet,In_red_Lane]}
                                send_email(filtered_text, violation_image_path, ', '.join(violation_type), email)
                            else:
                                # Update the violations for the license plate if new ones are found
                                current_violations = set(violations_dict[filtered_text]) #no helmet
                                new_violations = set(violation_type) #red lane,no helmet
                                updated_violations = list(current_violations | new_violations) #red_lane,nohelmt

                                # If new violations are found, update and send email
                                if updated_violations != violations_dict[filtered_text]:
                                    violations_dict[filtered_text] = updated_violations
                                    send_email(filtered_text, violation_image_path, ', '.join(updated_violations), email)

                            # Draw OCR text (English and Arabic) on the original frame
                            arabic_text = convert_to_arabic(filtered_text)
                            frame = draw_text_pil(frame, filtered_text, (x1, y2 + 30), font_path, font_size=30, color=(255, 255, 255))
                            frame = draw_text_pil(frame, arabic_text, (x1, y2 + 60), font_path, font_size=30, color=(0, 255, 0))

    # Write the processed frame to the output video
    out.write(frame)

# Release resources when done
cap.release()
out.release()

# Print violations dictionary at the end
print("Violations per license plate:")
for plate, violations in violations_dict.items():
    print(f"License Plate: {plate}, Violations: {', '.join(violations)}")



0: 384x640 (no detections), 26.6ms
Speed: 2.4ms preprocess, 26.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 25.0ms
Speed: 2.1ms preprocess, 25.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 24.6ms
Speed: 2.3ms preprocess, 24.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 24.6ms
Speed: 2.3ms preprocess, 24.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 24.6ms
Speed: 2.3ms preprocess, 24.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 24.6ms
Speed: 2.2ms preprocess, 24.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 24.7ms
Speed: 2.2ms preprocess, 24.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 MotorbikeDelivery, 24.6ms
Speed: 2.2ms preprocess, 24.6

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



0: 384x640 1 Helmet, 1 MotorbikeDelivery, 1 Person, 12.1ms
Speed: 2.2ms preprocess, 12.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 640x352 (no detections), 11.7ms
Speed: 1.5ms preprocess, 11.7ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 352)

0: 384x640 1 Helmet, 1 MotorbikeDelivery, 1 Person, 12.1ms
Speed: 2.2ms preprocess, 12.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 640x320 (no detections), 11.3ms
Speed: 1.4ms preprocess, 11.3ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 320)

0: 384x640 1 Helmet, 12.1ms
Speed: 2.2ms preprocess, 12.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Helmet, 11.3ms
Speed: 2.5ms preprocess, 11.3ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Helmet, 1 MotorbikeDelivery, 1 Person, 11.3ms
Speed: 3.7ms preprocess, 11.3ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 640x256

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Detection data for license plate 3321 AR inserted successfully.
Email found for license plate 3321 AR: fares5675@gmail.com
Email with attachment sent successfully!

0: 384x640 1 Helmet, 1 MotorbikeDelivery, 1 Person, 26.0ms
Speed: 2.2ms preprocess, 26.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 640x256 1 License_plate, 18.9ms
Speed: 1.3ms preprocess, 18.9ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 256)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Detection data for license plate 3321 AR inserted successfully.
Email found for license plate 3321 AR: fares5675@gmail.com

0: 384x640 1 Helmet, 1 MotorbikeDelivery, 1 Person, 12.1ms
Speed: 2.2ms preprocess, 12.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 640x256 (no detections), 12.1ms
Speed: 1.8ms preprocess, 12.1ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 256)

0: 384x640 1 Helmet, 1 Person, 12.1ms
Speed: 2.1ms preprocess, 12.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Helmet, 1 MotorbikeDelivery, 1 Person, 11.3ms
Speed: 2.2ms preprocess, 11.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 640x256 1 License_plate, 10.1ms
Speed: 1.1ms preprocess, 10.1ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 256)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



0: 384x640 1 Helmet, 1 MotorbikeDelivery, 1 Person, 12.1ms
Speed: 2.5ms preprocess, 12.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 640x288 (no detections), 11.0ms
Speed: 1.3ms preprocess, 11.0ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 288)

0: 384x640 1 Helmet, 1 MotorbikeDelivery, 1 Person, 12.1ms
Speed: 2.1ms preprocess, 12.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 640x256 1 License_plate, 9.8ms
Speed: 1.2ms preprocess, 9.8ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 256)

0: 384x640 1 Helmet, 1 MotorbikeDelivery, 1 Person, 12.1ms
Speed: 2.2ms preprocess, 12.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 640x288 (no detections), 10.8ms
Speed: 1.3ms preprocess, 10.8ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 288)

0: 384x640 1 Helmet, 1 MotorbikeDelivery, 1 Person, 12.1ms
Speed: 2.2ms preprocess, 12.1ms inference, 1.6ms postprocess per image a